#Next Word Prediction model

In [94]:
#Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pickle
import heapq

In [95]:
#loading the data to the variable
link = '1661-0.txt'
with open(link) as f:
    text= f.readlines()

In [96]:
#Reading and converting data to lower case
text = open(link).read().lower()
print('corpus length:', len(text))
text

corpus length: 581888


'\ufeff\nproject gutenberg\'s the adventures of sherlock holmes, by arthur conan doyle\n\nthis ebook is for the use of anyone anywhere at no cost and with\nalmost no restrictions whatsoever.  you may copy it, give it away or\nre-use it under the terms of the project gutenberg license included\nwith this ebook or online at www.gutenberg.net\n\n\ntitle: the adventures of sherlock holmes\n\nauthor: arthur conan doyle\n\nrelease date: november 29, 2002 [ebook #1661]\nlast updated: may 20, 2019\n\nlanguage: english\n\ncharacter set encoding: utf-8\n\n*** start of this project gutenberg ebook the adventures of sherlock holmes ***\n\n\n\nproduced by an anonymous project gutenberg volunteer and jose menendez\n\n\n\ncover\n\n\n\nthe adventures of sherlock holmes\n\n\n\nby arthur conan doyle\n\n\n\ncontents\n\n\n   i.     a scandal in bohemia\n   ii.    the red-headed league\n   iii.   a case of identity\n   iv.    the boscombe valley mystery\n   v.     the five orange pips\n   vi.    the man wi

In [97]:
#Importing RegexTeokenizer module from NLTK library
from nltk.tokenize import RegexpTokenizer

In [98]:
#Tokenizing the data and converting to tokens
tokenizer = RegexpTokenizer(r'\w+')
#lowering the case
wo = text.lower() 
words = tokenizer.tokenize(wo)
print(words)

['project', 'gutenberg', 's', 'the', 'adventures', 'of', 'sherlock', 'holmes', 'by', 'arthur', 'conan', 'doyle', 'this', 'ebook', 'is', 'for', 'the', 'use', 'of', 'anyone', 'anywhere', 'at', 'no', 'cost', 'and', 'with', 'almost', 'no', 'restrictions', 'whatsoever', 'you', 'may', 'copy', 'it', 'give', 'it', 'away', 'or', 're', 'use', 'it', 'under', 'the', 'terms', 'of', 'the', 'project', 'gutenberg', 'license', 'included', 'with', 'this', 'ebook', 'or', 'online', 'at', 'www', 'gutenberg', 'net', 'title', 'the', 'adventures', 'of', 'sherlock', 'holmes', 'author', 'arthur', 'conan', 'doyle', 'release', 'date', 'november', '29', '2002', 'ebook', '1661', 'last', 'updated', 'may', '20', '2019', 'language', 'english', 'character', 'set', 'encoding', 'utf', '8', 'start', 'of', 'this', 'project', 'gutenberg', 'ebook', 'the', 'adventures', 'of', 'sherlock', 'holmes', 'produced', 'by', 'an', 'anonymous', 'project', 'gutenberg', 'volunteer', 'and', 'jose', 'menendez', 'cover', 'the', 'adventures',

In [99]:
#Extracting the unique words using the np.unique function
unique_words = np.unique(words)
unique_word_index = dict((c, i) for i, c in enumerate(unique_words))

In [100]:
#Finding the previous and next words
LENGTH = 5
prev = []
next= []
for i in range(len(words) - LENGTH):
    prev.append(words[i:i + LENGTH])
    next.append(words[i + LENGTH])
#Printing the previous and next words
print("Previous :",prev[0])
print("Next :",next[0])

Previous : ['project', 'gutenberg', 's', 'the', 'adventures']
Next : of


In [101]:
#Creating the array with number of zeros
X = np.zeros((len(prev), LENGTH, len(unique_words)), dtype=bool)
Y = np.zeros((len(next), len(unique_words)), dtype=bool)

In [102]:
#Properly indexing the previous and next words
for i, each_words in enumerate(prev):
    for j, word in enumerate(each_words):
        X[i, j, unique_word_index[word]] = 1
    Y[i, unique_word_index[next[i]]] = 1

In [103]:
print(X[0][0])

[False False False ... False False False]


In [104]:
print(Y[0][0])

False


In [105]:
#Importing the Keras library for models and model operations
from keras.models import Sequential, load_model
from keras.layers import LSTM
from keras.layers.core import Dense, Activation
from keras.optimizers import RMSprop

In [106]:
#Selecting the Sequential model 
model = Sequential()
model.add(LSTM(128, input_shape=(LENGTH, len(unique_words))))
model.add(Dense(len(unique_words)))
#setting Softmax Activation function
model.add(Activation('softmax'))
#setting the RMSprop Optimizer
optimizer = RMSprop(learning_rate=0.01)

In [107]:
#Compiling the model
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

In [108]:
#Training the model.
history = model.fit(X, Y, validation_split=0.05, batch_size=128, epochs=2, shuffle=True).history

Epoch 1/2
811/811 [==============================] - 267s 325ms/step - loss: 6.3436 - accuracy: 0.0816 - val_loss: 7.1009 - val_accuracy: 0.1005
Epoch 2/2
811/811 [==============================] - 271s 334ms/step - loss: 5.6240 - accuracy: 0.1483 - val_loss: 7.8144 - val_accuracy: 0.1067


In [109]:
#Saving the model as h5 file
model.save('next_word_model.h5')
pickle.dump(history, open("history.p", "wb"))
#Loading the model
model = load_model('next_word_model.h5')
history = pickle.load(open("history.p", "rb"))

In [110]:
#Defining a function to prepare input for using in model
def prep(text):
    x = np.zeros((1, LENGTH, len(unique_words)))
    for a, word in enumerate(text.split()):
        print(word)
        x[0, a, unique_word_index[word]] = 1
    return x

In [111]:
#using the prep funtion we are transforming the data
prep("It is not a lack".lower())

it
is
not
a
lack


array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]])

In [112]:
#Defining a function called sample
def sample(preds, top_n=3):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds)
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    return heapq.nlargest(top_n, range(len(preds)), preds.take)

In [113]:
#Defining a function predict_completion to predict the data using model
def completion(text):
    original_text = text
    generated = text
    completion = ''
    indices_char={}
    while True:
        x = prep(text)
        preds = model.predict(x, verbose=0)[0]
        next_index = sample(preds, top_n=1)[0]
        next_char = indices_char[next_index]
        text = text[1:] + next_char
        completion += next_char
        if len(original_text + completion) + 2 > len(original_text) and next_char == ' ':
            return completion
def completions(text, n=3):
    x = prep(text)
    preds = model.predict(x, verbose=0)[0]
    next_indices = sample(preds, n)
    return [unique_words[idx] + completion(text[1:] + completion[idx]) for idx in next_indices]

In [115]:
def completions(text, n=3):
    if text == "":
        return("0")
    x = prep(text)
    preds = model.predict(x, verbose=0)[0]
    next_indices = sample(preds, n)
    return [unique_words[idx] for idx in next_indices]

In [116]:
q =  "Your life will never be the same again"
print("correct sentence: ",q)
seq = " ".join(tokenizer.tokenize(q.lower())[0:5])
print("Sequence: ",seq)
print("next possible words: ", predict_completions(seq, 5))


correct sentence:  Your life will never be the same again
Sequence:  your life will never be
your
life
will
never
be
next possible words:  ['in', 'the', 'so', 'a', 'at']
